In [20]:
import json
import os
import sys
sys.path.append('../')
from utils import get_score
import torch as t 
import numpy as np
from glob import glob
import pickle

In [16]:
from glob import glob
data_root="/data_ssd/zhihu/result/search_stack/"
files_path=glob(data_root+"*val.pth")
files_path.sort()
print len(files_path)

27


In [17]:
files_weight1=[]
initial_weight=[]
for file in files_path:
    if 'weight5' not in file:
        files_weight1.append(file)
        if 'MultiModel' in file:
            initial_weight.append(5)
        else:
            initial_weight.append(1)
print len(files_weight1)

22


In [18]:
for f,w in zip(files_weight1,initial_weight):
    print f,w

/data_ssd/zhihu/result/search_stack/CNNText_tmp0.4024_char_val.pth 1
/data_ssd/zhihu/result/search_stack/CNNText_tmp0.4109_word_val.pth 1
/data_ssd/zhihu/result/search_stack/DeepText0.4103_word_val.pth 1
/data_ssd/zhihu/result/search_stack/DeepText_word_0.410051_aug_val.pth 1
/data_ssd/zhihu/result/search_stack/FastText3_word_weight1_0.40853_val.pth 1
/data_ssd/zhihu/result/search_stack/LSTMText0.4031_char_val.pth 1
/data_ssd/zhihu/result/search_stack/LSTMText0.4119_word_val.pth 1
/data_ssd/zhihu/result/search_stack/LSTMText0.41368_aug_word_val.pth 1
/data_ssd/zhihu/result/search_stack/MultiCNNTextBNDeep_aug_char_0.38738_val.pth 1
/data_ssd/zhihu/result/search_stack/MultiModel2_0.42560_val.pth 5
/data_ssd/zhihu/result/search_stack/MultiModel2w2c_0.4213_val.pth 5
/data_ssd/zhihu/result/search_stack/MultiModel_0.4171859_val.pth 5
/data_ssd/zhihu/result/search_stack/MultiModel_0.41987_word_val.pth 5
/data_ssd/zhihu/result/search_stack/MultiModel_0.42169_val.pth 5
/data_ssd/zhihu/result/se

In [21]:
model_num=10
probs=[t.load(r).float() for r in files_weight1[:model_num]]
test_data_path='/home/a/code/pytorch/zhihu/ddd/val.npz'
index2qid = np.load(test_data_path)['index2qid'].item()
label_path="/home/a/code/pytorch/zhihu/ddd/labels.json"
with open(label_path) as f: 
      labels_info = json.load(f)
qid2label = labels_info['d']
true_labels = [qid2label[index2qid[2999967-200000+ii]] for ii in range(200000)]
del labels_info
del qid2label
del index2qid

In [22]:
def target(args):
    r=0
    for r_,k_ in enumerate(args):
        if r_<model_num:
            r +=k_*probs[r_]
        else:
            tmp=t.load(files_path[r_]).cuda().float()
            r=r+k_*tmp.cpu()
    result = r.topk(5,1)[1]
    predict_label_and_marked_label_list = [[_1,_2] for _1,_2 in zip(result,true_labels)]
    score,_,_,_ = get_score(predict_label_and_marked_label_list)
    print (args,score,_)#list_space = [hp.uniform('a',0,1),hp.uniform('b',0,1)]
    return -score

In [ ]:
max_evals=100
from hyperopt import hp, fmin, rand, tpe, space_eval
list_space = [hp.normal('a'+str(rr),1,0.2) for rr in range(len(files_path))]
from hyperopt import Trials
trials_to_keep=Trials()
best = fmin(target,list_space,algo=tpe.suggest,max_evals=max_evals, trials = trials_to_keep)
output = open('trials_to_keep__weight1_model'+'.pkl', 'wb')
pickle.dump(trials_to_keep, output)

((0.5767518711990203, 1.0535449863681772, 1.387628321556626, 0.8174540261504482, 0.46505220685010107, 0.84297799521111, 0.6365171155277725, 1.1619862524737483, 1.1629124711170251, 0.794240785899519, 1.3500216705996364, 0.7161582546569272, 0.8632489643123853, 1.0658517932708935, 0.7141997070591544, 1.1636288618440052, 0.8801467587212355, 0.9337439124613484, 0.7588341161043034, 1.0673229231836008, 0.9405260442990232, 1.0030074530136603, 1.233610926900131, 1.072457858205536, 0.8331957511909095, 0.8314988827650005, 1.3867110853019984), 0.4280465816350192, [118235, 66455, 43228, 29651, 21732])
((0.9564174440498158, 1.0489559058574427, 0.7518749608094916, 1.0320464666633937, 0.9422874525955014, 1.0165090602024747, 1.0795518519765972, 0.8555932208868637, 1.2167276710281532, 1.419475183806352, 0.8838074782881737, 0.9468634334505386, 0.9691711235479938, 1.2110952652578792, 0.7660644586038812, 1.0678274437017292, 1.2004882120913785, 0.5873016262887354, 0.9972641287871216, 1.232087128048666, 0.59

In [ ]:
best